In [25]:
import pandas as pd
import nltk
from nltk import tokenize
from utils import note_taker
import re
import json

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
df = pd.read_csv('legislacao.csv', sep = '\t')
df.head()

,tipoDocumento,data,url,title,description,identifier,entity_legal
0,Decreto,1987-09-24,https://www.lexml.gov.br/urn/urn:lex:br:federa...,"['Decreto no 94.949, de 24 de Setembro de 1987...","['Abre, a Encargos Gerais da União - Recursos...",895290,Decreto no 94.949
1,Decreto,1987-09-24,https://www.lexml.gov.br/urn/urn:lex:br:federa...,"['Decreto no 94.950, de 24 de Setembro de 1987...",['Abre ao Ministério da Indústria e do Comé...,895291,Decreto no 94.950
2,Decreto,1987-08-11,https://www.lexml.gov.br/urn/urn:lex:br:federa...,"['Decreto no 94.764, de 11 de Agosto de 1987',...","['Altera o Decreto no 88.351, de 1o de junho d...",895292,Decreto no 94.764
3,Decreto,1987-08-06,https://www.lexml.gov.br/urn/urn:lex:br:federa...,"['Decreto no 94.740, de 6 de Agosto de 1987', ...",['Abre ao Ministério da Reforma e do Desenvol...,895293,Decreto no 94.740
4,Decreto,1987-09-24,https://www.lexml.gov.br/urn/urn:lex:br:federa...,"['Decreto no 94.951, de 24 de Setembro de 1987...",['Outorga concessão à RÁDIO PIONEIRA DE FOR...,895294,Decreto no 94.951


In [13]:
tipo_documento = df.tipoDocumento.value_counts()
tipo_documento

Decreto                                                32240
Decreto Legislativo                                    11836
Medida Provisória                                      7200
Lei                                                     7037
Decreto-Lei                                              332
Lei Complementar                                         134
Emenda Constitucional                                    119
Lei Delegada                                               2
Constituição                                             1
Ato das Disposições Constitucionais Transitórias        1
Name: tipoDocumento, dtype: int64

In [14]:
description_arr = df.description.to_list()
            

In [1]:
train_docs = []
test_docs = []

amount_text = 0
amount_train = 0
amount_test = 0

lines = []
words = []
chars = []
df_note = pd.DataFrame({'named_entities': []})
for description in description_arr:
    named_entities_arr = []

    if type(description) is not float:
        text = description.split("'")[1] if len(description.split("'")) > 1 else description
        
            
        named_entities_arr.extend(re.findall(r'Lei\sComplementar\sno\s\d+.\d+|Lei\sComplementar\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'Lei\sDelegada\sno\s\d+.\d+|Lei\sDelegada\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'\sLei\sno\s\d+.\d+|Lei\sno\s\d+ ', text))
        named_entities_arr.extend(re.findall(r'Decreto-Lei\sno\s\d+.\d+|Decreto-Lei\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'Decreto\sLegislativo\sno\s\d+.\d+|Decreto\sLegislativo\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'\sDecreto\sno\s\d+.\d+|Decreto\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'Medida\sProvisória\sno\s\d+.\d+|Medida\sProvisória\sno\s\d+', text))
        named_entities_arr.extend(re.findall(r'Emenda\sConstitucional\sno\s\d+.\d+|Emenda\sConstitucional\sno\s\d+', text))

        if named_entities_arr:
            amount_text += 1
            corpus, text_tokens = note_taker(text, named_entities_arr)

            if amount_text < 9: 
                train_docs.extend(corpus).append(' ')
                amount_train += 1

            elif amount_text < 11:
                test_docs.extend(corpus).append(' ')
                amount_test += 1

            else: amount_text = 0

            words.extend(text_tokens)
            df_note = df_note.append(named_entities_arr, ignore_index=True)

with open('dataset/train.txt', 'w') as arquivo:
    for train in train_docs: arquivo.write(f'{train}\n')
               
with open('dataset/test.txt', 'w') as arquivo:
    for test in test_docs: arquivo.write(f'{test}\n')

with open('dataset/words.txt', 'w') as words_arq:
    for word in set(words): 
        words_arq.write(f'{word}\n')
        chars.extend(list(word))

with open('dataset/chars.txt', 'w') as chars_arq:
    for char in set(chars): chars_arq.write(f'{char}\n')

df_note.to_csv('dataset/note_taker_automatic_info.csv', sep = '\t')

NameError: name 'pd' is not defined

In [17]:
print(f'train: {amount_train}\ntest: {amount_test}')

train: 6719
dev: 839
test: 839


In [20]:
df_train = pd.read_csv('dataset/train.txt', sep=" ", header=None)
df_train.columns = ['token', 'entity']

df_test = pd.read_csv('dataset/test.txt', sep=" ", header=None)
df_test.columns = ['token', 'entity']

In [34]:
df_train.entity.value_counts()

O          288367
I-LEI        9351
I-DEC        6292
B-LEI        4675
B-DEC        3145
I-DLEI        790
B-DLEI        395
I-LCOMP       390
I-None        381
B-LCOMP       130
B-None        127
I-DLEG        111
I-EC           57
B-DLEG         37
B-EC           19
I-LDEL          9
B-LDEL          3
Name: entity, dtype: int64

In [35]:
df_dev.entity.value_counts()

O          38167
I-LEI       1248
I-DEC        758
B-LEI        624
B-DEC        379
I-DLEI       100
I-LCOMP       72
I-None        63
B-DLEI        50
B-LCOMP       24
B-None        21
I-EC          18
I-DLEG        15
B-EC           6
B-DLEG         5
Name: entity, dtype: int64

In [36]:
df_test.entity.value_counts()

O          37565
I-LEI       1199
I-DEC        795
B-LEI        599
B-DEC        397
I-DLEI       104
I-None        57
B-DLEI        52
I-LCOMP       45
B-None        19
B-LCOMP       15
I-DLEG         9
I-EC           3
I-LDEL         3
B-DLEG         3
B-EC           1
B-LDEL         1
Name: entity, dtype: int64